# Practica 5
El objetivo de esta práctica es el de entrenar un clasificador basado en red neuronal fully connected empleando las características de cada audio obtenidas al final de la práctica 4.

Para la implementación y entrenamiento de la red neuronal emplearemos la libreria [pytorch](https://pytorch.org/). Pytorch tiene un manejo de tensores muy similar a numpy pero tiene dos ventajas importantes:
* Permite la diferenciación (cálculo del gradiente) de forma automática.
* Permite el uso de la GPU de forma muy sencilla, lo cual puede acelerar los tiempos de entrenamiento e inferencia de forma considerable.

La mejor forma de introducirse en el manejo de Pytorch es consultar algunos de los tutoriales de ayuda disponibles en: [Tutoriales](https://pytorch.org/tutorials/).

## Diferenciación automática

Quizás esta es la carácterística más importante que hace de Pytorch una herramienta tan potente. 

Consideremos que $X \in \mathbb{R}^n$ es un vector de características y  $W \in \mathbb{R}^n$ es un vector con unos pesos y calculamos la siguiente expresión:abs

$y = W^T * X$ 

Es decir el producto escalar de ambos vectores. Para calcular el gradiente $\nabla y = [\delta y/ \ \delta x_0, \delta y/ \ \delta x_1, \ldots \delta y/ \ \delta x_{n-1} ] \$, lo podemos hacer de la siguiente forma:



In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pytest
import UPVlog


ModuleNotFoundError: No module named 'UPVlog'

In [ ]:
X = np.random.randn(5) # vector de entrada aleatorio
W = np.random.randn(5) # vector de pesos aleatorio
y = np.dot(X, W) # producto escalar

# Convertimos de numpy a torch
Xt = torch.from_numpy(X)
Wt = torch.from_numpy(W)
yt = torch.dot(Xt,Wt)

print(f"{X=}")
print(f"{Xt=}")
print("-"*10)
print(f"{W=}")
print(f"{Wt=}")
print("-"*10)

print(f"resultado con numpy = f{y}")
print(f"resultado con torch = f{yt}")

Como vemos el resultado es identico empleando numpy o Pytorch. Además el paso de tensores de una librería a la otra es inmediato. 

Si quisieramos calcular el gradiente empleando numpy, deberíamos programar explicitamente la función. Este paso podría ser relativamente complejo en el caso de que la función fuera relativamente compleja (además de que sería bastante común la introducción de errores en la programación). 

Sin embargo, Pytorch nos ofrece una forma mucho más flexible de calcular los gradientes. Para ello: 
* Debemos indicar que tensores requieren el cálculo del gradiente.
* Asegurarnos de que la variable sobre la que se cálcula el gradiene (en nuestro caso `y`, sea un escalar

Entonces podemos hacer lo siguiente:




In [ ]:
# indicamos que queremos calcular el gradiente respecto de Wt
Wt.requires_grad = True


y2 = torch.dot(Wt, Xt) # realizamos la operacion que deseemos

y2.backward() # calculamos las derivadas y aplicamos la regla de la cadena de forma recursiva


print(f"{y2=}") # el esultado es igual que antes

print(f"El gradiente vale: {Wt.grad}")



Como vemos el código anterior ha calculado el gradiente de forma automática. En este caso tan sencillo, el gradiente lo podemos calcular de forma exacta:abs

$\nabla y = X$

Comprueba que efectivamente el valor del gradiente coincide con el vector `X`.

Ahora vamos a añadir una función un poco más compleja que la anterior. La función correspondería con la evaluación de la función de coste de un clasificador lineal binario. Que tendría un grafo de operación como el de la siguiene imagen:

![computational graph](figs/comp-graph.png)

El código para implementar el grafo anterior sería. 


In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

loss.backward() # calculamos gradientes

print(f"Gradiente de w: {w.grad}") # tiene tantos elementos como w
print(f"Gradiente de b: {b.grad}") # tiene tantos elementos como w



## Definición del dataset/dataloader

Una vez vista la capacidad de Pytorch para calcular de forma automáticamente los gradientes. Pasaremos a la gestión de los datos. 

Para ello emplearemos las clases Dataset y Dataloader (ver este [enlace](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) para mas información).

### Dataset

El Dataset es una clase que permite organizar todos los datos disponibles para entrenar o validar. Cuando queremos crear un dataset específico, crearemos una subclase en la que reimplementaremos los tres metodos siguientes:

* `def __init__(self, *args, **kwargs)`: función para indicar como debe obtener los datos
* `def __len__(self)`: función para indicar cuantos datos tenemos
* `def __getitem__(self, idx)`: Función que devuelve el elemento idx (numero entero) del dataset.

El método más interesante de los anteriores es `__getitem__` que devuelve el ejemplo que ocupa la posición `idx`de train/validación.

Vamos a crear un dataset con los vectores de características de los audios.  Pero antes de eso vamos a cargar y normalizar 
las características que tenemos precalculadas de todos los audios proporcionados por los alumnos.

In [ ]:
train_data = np.load('audio_features_train.npz')
features_train = train_data['features']
labels_train = train_data['labels']
print("features_train shape", features_train.shape)
print("labels_train shape", labels_train.shape)


In [ ]:
#repetimos para los de validacion
valid_data = np.load('audio_features_val.npz')
features_valid = valid_data['features']
labels_valid = valid_data['labels']
print("features_valid shape", features_valid.shape)
print("labels_valid shape", labels_valid.shape)

In [ ]:
# Calculamos media y std del train set
# Es importante keepdims para poder hacer broadcasting
m_train = features_train.mean(axis=0, keepdims = True)
s_train = features_train.std(axis=0, keepdims = True)

#normalizamos las features en ambos casos con la media y std de train
features_train_norm = (features_train - m_train) / s_train
features_valid_norm = (features_valid - m_train) / s_train

In [ ]:
from torch.utils.data import Dataset

class P5Dataset(Dataset): # heredamos de la clase Dataset 
    def __init__(self, features, labels):
        #Simplemente guardamos las features y las labels
        assert features.shape[1] == 85 #Comprobaciones
        assert features.shape[0] == labels.shape[0]
        self.features = features.astype(np.float32) # reducimos precision a float32
        self.labels = labels
    def __len__(self):
        return self.labels.shape[0] # numero de ejemplos del dataset
    def __getitem__(self, idx):
        # devolvemos features, y label del ejemplo idx
        return self.features[idx], self.labels[idx]

In [ ]:
train_dataset = P5Dataset(features_train_norm, labels_train)
valid_dataset = P5Dataset(features_valid_norm, labels_valid)

Vamos a pintar un ejemplo para comprobar:

In [ ]:
sample_idx = 100

features, label = train_dataset[sample_idx]

plt.plot(features)
plt.xlabel('features')
plt.title(f'label = {label}')



Podemos comprobar como tras la normalización los valores de todas las características aparecen en un rango dinámico parecido. 

### Dataloader

El Dataset es util para organizar los datos, pero durante el entrenamiento de una red necesaria es necesario ir tomando grupos aleatorios de ejemplos de entrenamiento para formar los batches. En ocasiones la obtención de un ejemplo a partir del Dataset puede tardar cierto tiempo (por ejemplo en el caso de que los datos de entrenamiento sean imagenes). Por ello, la clase Dataloader se encarga de:
* Seleccionar los ejemplos que van a formar un batch
* Controlar las todas las iteraciones que serán necesarias para formar un epoch
* Paralelizar la extracción de datos del Dataset

Veamos como definir un dataset y como realizar un bucle para realizar un epoch:


In [ ]:
from torch.utils.data import DataLoader

#necesitamos dataset, batch_size y si queremos shuffle (orden aleatorio)
batch_size = 512
train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


In [ ]:
from tqdm import tqdm # bucles bonitos

for batch_feat, batch_labels in tqdm(train_dataloader): # genera un iterador
    pass # en este bucle no hacemo nada todavia solo queremos ver el iterador 

En el resultado anterior podemos ver:
* Que hemos completado una pasada por todos los datos (epoch)
* El numero de batches que hemos empleado
* La velocidas (iteraciones/s)

Más abajo emplearemos los dataloaders para iterar por todos los ejemplos de entrenamiento y validación, pero de momento vamos a pasar a definir la red neuronal.  

# Primera red neuronal

Una vez definidos los datasets y dataloaders para la gestión de los datos, vamos a definir la red neuronal. 

## Módulos en Pytorch

Pytorch plantea una estructura modular para construir las redes neuronales. El bloque básico se llama [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html), este módulo implementa los siguientes métodos:
* `__init__`: Es el constructor donde se indica como inicializar
* `forward`: Este es el método donde se indica lo que debe hacer el módulo
* `backward`: Para calcular los gradientes

En la práctica construiremos nuevos módulos ensamblando módulos más simples. Para ello únicamente reimplementaremos los métodos `__init__` y `__forward__`, porque el método backward está gestionado por la clase padre `nn.Module`. 

Veamos un ejemplo, primero definiremos un módulo lineal (empleado en las capas de las redes fully connected)


In [ ]:
import torch
from torch import nn

input_dim = 10
out_dim = 5

# solo hay que indicar el número de neuronas de entrada y salida
# La gestión de las variables que contienen los parametros la realiza
# el módulo automáticamente
mi_modulo = nn.Linear(input_dim, out_dim)

#ejemplo de uso
x = torch.randn(1,input_dim) # tiene que ser array bidimensional
# el numero de filas indica el numero de ejemplos que procesamos simultáneamente

y = mi_modulo(x) ## automticamente llama al metodo forward()

print("La dimension 1 deberia ser out_dim:",y.shape)
assert y.shape[1] == 5



 ## Nuevos módulos
Pytorch proporciona múltitud de módulos básicos que podemos combinar muy fácilmente para crear nuevos. En este ejemplo construiremos un nuevo modulo que:
* Incluye capa lineal
* Normaliza (batch norm)
* Aplica no linealidad tipo ReLU

Todos los nuevos modulos deben heredar de `nn.Module`:
 

In [ ]:

class Capa(nn.Module): 
    def __init__(self, input_dim, out_dim): 
        super().__init__() #importante llamar al constructor del padre lo primero
        
        self.ff = nn.Linear(input_dim, out_dim) # capa lineal
        self.bn = nn.BatchNorm1d(out_dim)
        self.relu = nn.ReLU(inplace=True)


    def forward(self, X): # aqui definimos lo que hara el nuevo modulo
        y = self.ff(X)
        y = self.bn(y)
        y = self.relu(y)
        return y


Vamos a probar como usar nuestro nuevo modulo:

In [ ]:
input_dim = 85
out_dim = 10 

capa = Capa(input_dim,out_dim)

x = torch.randn(20,85) # 20 ejemplos random

y = capa(x) # igual que antes, llama a forward

print("Dimensiones y =", y.shape)

assert torch.all(y >= 0) # tras relu todos son >= 0



Una vez visto como generar un modulo ensablando varios vamos a construir nuevos modulos. 

Comenzaremos modificando el modulo `Capa`, que acabamos de construir para incluir una capa de [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) entre el `Batchnorm` y `ReLU`



In [ ]:
#Este módulo incluirá dropout en el caso de que 
# p_dropout > 0.0. Por defecto no incluye dropout

class Capa(nn.Module): 
    def __init__(self, input_dim, out_dim, p_dropout: float = 0.0): 
        super().__init__() #importante llamar al constructor del padre lo primero
        
        self.ff = nn.Linear(input_dim, out_dim) # capa lineal
        self.bn = nn.BatchNorm1d(out_dim)
        #Defina aqui el modulo dropout si p_dropout > 0
        self.p_dropout = p_dropout # guardamos el valor para saber si hay que aplicarlo durante el forward
        
        # YOUR CODE HERE
        if p_dropout > 0:
            self.dropout = nn.Dropout(p_dropout)
        else:
            self.dropout = None
 
        self.relu = nn.ReLU(inplace=True)


    def forward(self, X): # aqui definimos lo que hara el nuevo modulo
        y = self.ff(X)
        y = self.bn(y)
        # aplique aqui dropout si self.p_dropout > 0
        
        # YOUR CODE HERE
        if self.dropout is not None:
            y = self.dropout(y)        
        
        y = self.relu(y)
        return y


In [ ]:
# Comprobación
my_logger.test("Capa")
capa = Capa(85,10,p_dropout=0.2)

assert len([m for m in capa.modules() if isinstance(m, nn.Dropout)]) == 1, "no ha incluido el modulo dropout"
dropout_module = [m for m in capa.modules() if isinstance(m, nn.Dropout)][0]
assert dropout_module.p == 0.2, "Ajuste la probabilidad del modulo de dropout"
my_logger.success("Capa",1.0)


Una red neuronal en Pytorch es simplemente objeto de la clase `nn.Module` más, pero que abarca desde la entrada hasta la salida. El siguiente ejemplo muestra la estructura de la red neuronal que emplearemos en esta práctica.

En este caso, nuestra Red hereda de la clase [`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html).
Simplemente tenemos que definir una lista de modulos que componen nuestra red para inicializar `nn.Sequential`. Lo interesante, es que no hay que redefinir el método `forward` porque ya está definido en la clase padre `nn.Sequential`.


In [ ]:
class Red(nn.Sequential):
    def __init__(self, input_dim=85, hidden_dim = [4,], out_dim = 10, p_dropout = 0.0):
        assert len(hidden_dim)>0, "at least one hidden layer"
        layers = [] #aqui guardamos una lista con todas las capas
        #first layer
        layers.append(Capa(input_dim,hidden_dim[0], p_dropout=p_dropout))

        #Output layer
        layers.append(nn.Linear(hidden_dim[-1], out_dim))
        super().__init__(*layers) #inicializamos la clase padre con la lista de capas
                
                      
        

In [ ]:
mired = Red()
print(mired)


La clase anterior crea una red neuronal con una capa oculta de 4 neuronas. El número de neuronas de la capa oculta viene definido en la variable `hidden_dim`. Como `hidden_dim` es una lista podríamos añadir nuevas capas ocultas añadiendo más elementos a la lista `hidden_dim`.

Complete el código siguiente para que si `hidden_dim` tiene N elementos cree N capas ocultas cada una con el número de neuronas correspondiente.


In [ ]:
class Red(nn.Sequential):
    def __init__(self, input_dim=85, hidden_dim = [4,], out_dim = 10, p_dropout = 0.0):
        assert len(hidden_dim)>0, "at least one hidden layer"
        layers = [] #aqui guardamos una lista con todas las capas
        #first layer
        layers.append(Capa(input_dim,hidden_dim[0], p_dropout=p_dropout))
        # Inserte el código para generar más capas ocultas si el número de elementos de hidden_dim > 1
        
        # YOUR CODE HERE
        for i in range(1, len(hidden_dim)):
            layers.append(Capa(hidden_dim[i-1], hidden_dim[i], p_dropout=p_dropout))
             
        #Output layer
        layers.append(nn.Linear(hidden_dim[-1], out_dim))
        super().__init__(*layers) #inicializamos la clase padre con la lista de capas
                
                      

In [ ]:
# Comprobacion
my_logger.test("Multicapa")


red = Red(input_dim = 85, hidden_dim=[5,7,5])
print(red)
assert len(list(red.children())) == 4, "No has creado el número de capas ocultas correctamente"
capas = list(red.children())
assert capas[0].ff.out_features == 5, "la primera capa tiene 5 neuronas"
assert capas[1].ff.out_features == 7, "la segunda capa tiene 7 neuronas"
assert capas[2].ff.out_features == 5, "la tercera capa tiene 5 neuronas"
assert capas[3].out_features == 10, "la capa de salida tiene 10 neuronas"
assert capas[0].ff.in_features == 85, "la primera capa tiene 5 neuronas"
assert capas[1].ff.in_features == 5, "la segunda capa tiene 7 neuronas"
assert capas[2].ff.in_features == 7, "la tercera capa tiene 5 neuronas"
assert capas[3].in_features == 5, "la capa de salida tiene 10 neuronas"

my_logger.success("Multicapa",2.0)


# Entrenando la red

En el fichero `train_net` se le proporciona las funciones para entrenar la red neuronal empleando la red y dataloaders que acaba de crear. 

Si examina el código mientras entrena las redes observará lo siguiente:
* Función `train_epoch`, es la encargada de realizar una pasada (epoch) por los datos de entrenamiento e ir actualizando los pesos mediante el método de gradiente
* Funtion `valid_epoch`, realiza una pasada sobre los datos de validación y obtiene las metricas de la red. En este caso los pesos no se actualizan y se mantienen fijos durante todo el proceso.
* Función `train`, alterna llamadas a `train_epoch`y `valid_epoch` durante un numero de epochs. Guarda los resultados de las métricas durante el proceso de entrenamiento.

A continuación se muestra como entrenar la red para un caso sencillo:


In [ ]:
from train_net import train
import torch.optim as optim


# Define learning rate
learning_rate = 0.01
batch_size = 512

# Define network
red1 = Red(input_dim = 85,hidden_dim=[4], p_dropout=0)

print(red1)

# Create Adam optimizer
#optimizer = optim.SGD(red1.parameters(), lr=learning_rate, momentum =0.9)
optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


num_epochs = 5
train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red1, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

# Pruebas de entrenamiento

En esta parte vamos a ir realizando distintos entrenamientos cambiando diferentes hiperparametros. 

### Efecto de la tasa de aprendizaje

En este apartado veremos el efecto de subir excesivamente el valor de la tasas de aprendizaje (learning rate).
Empleando la siguiente configuración:

* Learning rate = 10
* Numero de capas ocultas 1, neuronas 4
* Epochs = 10
* Batch size: 512
* No dropout


In [ ]:
# Completa con los valores que se indica para este apartado.
learning_rate = 10
batch_size = 512
num_epochs = 10
# Define network
red = Red(hidden_dim=[4], p_dropout=0.0)

# Create Adam optimizer
#optimizer = optim.SGD(red1.parameters(), lr=learning_rate, momentum =0.9)
optimizer = optim.Adam(red.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobación 
my_logger.test("efecto lr")
# En este caso con un learning rate tan grande no converje y el accuraccy es muy bajo
assert np.max(val_acc) == pytest.approx(0.12, abs=2e-2)
my_logger.success("efecto lr",1)


### Aumentar el número de epocas
Veamos ahora el efecto de poner más o menos epocas en el entrenamiento empleando 
el learning rate del primer apartado. 

* Learning rate = 0.01
* Numero de capas ocultas 1, neuronas 4
* Epochs = 10
* Batch size: 512
* Sin dropout


In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 512
num_epochs = 10
# Define network
red = Red(hidden_dim=[4], p_dropout=0.0)

# Create Adam optimizer
#optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
optimizer = optim.Adam(red.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto num_epoch")

# En este caso vemos que el algoritmo ya comienza a converger y que 
# El accuracy es mucho más alto, aunque da la sensacion de que con más
# epochs seguiria subiendo 
assert np.max(val_acc) == pytest.approx(0.68, abs=1e-1)
my_logger.success("efecto num_epoch",1)


Ahora repetiremos pero poniendo más epocas:
* Learning rate = 0.01
* Numero de capas ocultas 1, neuronas 4
* Epochs = 100
* Batch size: 512
* Sin dropout



In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 512
num_epochs = 100
# Define network
red = Red(hidden_dim=[4], p_dropout=0.0)

# Create Adam optimizer
#optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
optimizer = optim.Adam(red.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto muchos num_epoch")

# Al aumentar los epoch, hemos conseguido mejorar unque da la impresion de que
# pasado un tiempo satura el aprendizaje. El modelo que tenemos es demasiado pequeño 
# para esta tarea.
assert np.max(val_acc) == pytest.approx(0.72, abs=4e-2)
my_logger.success("efecto muchos num_epoch",1)


## Efecto del tamaño de batch
Veremos ahora el efecto de tomar cambiar `batch_size`, mire los resultados
de apartados anteriores para comparar.

* Learning rate = 0.01
* Numero de capas ocultas 1, neuronas 4
* Epochs = 70
* Batch size: 32
* Sin dropout



In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 32
num_epochs = 70
# Define network
red = Red(hidden_dim=[4], p_dropout=0.0)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto batchsize")
# El tiempo de entrenamiento es mucho mayor ahora ya que se realizan muchos mas pasos de gradiente
assert np.max(val_acc) == pytest.approx(0.73, abs=2e-2)
my_logger.success("efecto batchsize",0.5)


Ahora subimos el tamaño del batch:
* Learning rate = 0.01
* Numero de capas ocultas 1, neuronas 4
* Epochs = 70
* Batch size: 256
* Sin dropout


In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 256
num_epochs = 70
# Define network
red = Red(hidden_dim=[4], p_dropout=0.0)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto batch_size grande")

# El el resultado ha sido similar pero con menos iteraciones (más rapido)
assert np.max(val_acc) == pytest.approx(0.717, abs=5e-2)
my_logger.success("efecto batch_size grande",0.5)


Ahora subimos el tamaño del batch a un valor extremadamente grande:
* Learning rate = 0.01
* Numero de capas ocultas 1, neuronas 4
* Epochs = 70
* Batch size: 1024
* Sin dropout


In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 1024
num_epochs = 70
# Define network
red = Red(hidden_dim=[4], p_dropout=0.0)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto batch_size muy grande")

# El tamaño de batch es tan grande que con 70 epochs aun no ha terminado de entrenar porque tenemos pocas iteraciones
# Si repetieramos la prueba con más epochs llegariamos a resultados similares a los anteriores
assert np.max(val_acc) == pytest.approx(0.66, abs=4e-1)
my_logger.success("efecto batch_size muy grande",0.5)


**NOTA**: De los apartados anteriores parece deducirse que al aumentar el tamaño del batch el resultado empeora. Pero es una conclusion no del todo adecuada. Tenga en cuenta que al aumentar el tamaño del batch hay menos iteraciones del optimizador por epoch (los pesos de la red se actualizan menos veces) y lo que ocurre a la vista de las gráficas anteriores es que quizas deberiamos aumentar el número de epochs en este último caso para que le de tiempo a la red a converger.

## Cambio estructura de la red. 

Ahora probaremos el efecto de variar el número de neuronas y de capas. Comenzaremos aumentando el número de capas.

Ahora subimos el tamaño:
* Learning rate = 0.01
* Numero de capas ocultas 2, neuronas 4,4
* Epochs = 100
* Batch size: 512
* Sin dropout




In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 512
num_epochs = 100
# Define network
red = Red(hidden_dim=[4, 4], p_dropout=0.0)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto pocas neuronas")

# Resultado similar al anterior, tenemos muy pocas neuronas
assert np.max(val_acc) == pytest.approx(0.7, abs=2e-1)
my_logger.success("efecto pocas neuronas",0.5)


Ahora subimos el tamaño:
* Learning rate = 0.01
* Numero de capas ocultas 1, neuronas 40
* Epochs = 100
* Batch size: 512
* Sin dropout


In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 512
num_epochs = 100
# Define network
red = Red(hidden_dim=[40], p_dropout=0.0)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto mas neuronas")

# ahora si que vemos una mejora significativa en validacion
# Tambien observamos que el modelo tiene mucha más capacidad. En training va mucho mejor que en validacion
assert np.max(val_acc) == pytest.approx(0.838, abs=2e-2)
my_logger.success("efecto mas neuronas",0.5)


Ahora subimos el tamaño y el número de capas:
* Learning rate = 0.01
* Numero de capas ocultas 2, neuronas 40, 40
* Epochs = 100
* Batch size: 512
* Sin dropout


In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 512
num_epochs = 100
# Define network
red = Red(hidden_dim=[40, 40], p_dropout=0.0)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto mas capas y neuronas")

# el resultado es similar al anterior, pero observamos overfitting el modelo es muy muy potente
# El val_loss comienza a aumentar despues del epoch 20
assert np.max(val_acc) == pytest.approx(0.83, abs=2e-2)
my_logger.success("efecto mas capas y neuronas",0.5)


## Regularizacion

Para reducir el overfitting del caso anterior vamos a añadir regularización. En este caso vamos a añadir dropout 

Ahora subimos el tamaño y el número de capas:
* Learning rate = 0.01
* Numero de capas ocultas 2, neuronas 40, 40
* Epochs = 200
* Batch size: 512
* p_dropout = 0.25



In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 512
num_epochs = 200
# Define network
red = Red(hidden_dim=[40, 40], p_dropout=0.25)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cpu')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto dropout")

# el dropout reduce el overfitting y ademas mejora el resultado de validacion
assert np.max(val_acc) == pytest.approx(0.854, abs=2e-2)
my_logger.success("efecto dropout",0.5)


El ultimo modelo aumentaremos el numero de neuronas y mantendremos el dropout, ademas de subir el número de epoch
* Learning rate = 0.01
* Numero de capas ocultas 2, neuronas 100, 100
* Epochs = 400
* Batch size: 512
* p_dropout = 0.25




In [ ]:
# Define learning rate
learning_rate = 0.01
batch_size = 512
num_epochs = 400
# Define network
red = Red(hidden_dim=[100, 100], p_dropout=0.25)

# Create Adam optimizer
optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
#optimizer = optim.Adam(red1.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


train_dataloader = DataLoader(train_dataset, 
                              batch_size = batch_size,
                              shuffle= True )


val_dataloader =  DataLoader(valid_dataset, 
                              batch_size = batch_size,
                              shuffle= False,
                              drop_last=False) # 

#En train si el numero de muestras del dataset no es multiplo del batch_size se desprecian unas pocas cada epoch
#Para validar el ultimpo batch tendra seguramente menos muestras

train_losses, val_losses, train_acc, val_acc = train(red, loss_fn, train_dataloader,val_dataloader, optimizer, num_epochs, device='cuda')

fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

In [ ]:
# Comprobacion 
my_logger.test("efecto dropout y red mas grande")

# finalmente con dropout podemos subir la complejidad del modelo y aun mejora un poco mas
assert np.max(val_acc) == pytest.approx(0.861, abs=2e-2)
my_logger.success("efecto dropout y red mas grande",0.5)


# Prueba con grabación

Tomaremos el modelo del apartado anterior para hacer nuestras pruebas con audios grabados
al instante.

Lo primero que haremos es  completar las funciones con el código que **realizaste en la práctica 4** para la extracción de características


In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio
import librosa

In [ ]:
#  COPIA EL CODIGO DE LA PRACTICA 4 en las siguientes funciones

import numpy as np
import librosa
from utils import detect_speech

def spectral_centroid(S,f):
    """ Arguments:
        S: Espectrogram (complex matrix obtained using stft)
        f: frequencies vector, vectorf with the corresponding frequency of each row of S
        Returns:
           Vector with the spectral centroid for all frames

        NOTE: use of librosa.feature.spectral_centroid is not allowed
    """
    eps = 1e-10
    S = abs(S) # Tomamos valor absoluto

    numerador = np.sum(S*f[:, None], axis=0)
    denominador = np.sum(S, axis=0) + eps
    
    sc = numerador/denominador # Guarde aqui el vector centroide espectral
    
    return sc
    
def spectral_flux(S):
    S = np.abs(S) # nos aseguramos de tomar la magnitud
    
    flux = np.zeros(S.shape[1])

    #diferencia de cada trama consecutiva
    diff = S[:, 1:] - S[:, :-1]

    #
    flux[1:] = np.sum(diff**2, axis=0)
    
    return flux

def spectral_spread(S, f, centroid = None):
    S = np.abs(S)
    eps = 1e-10
    if centroid is None:
        centroid = spectral_centroid(S,f)

    centroid = centroid.reshape(1,-1)
    f = f.reshape(-1,1)
    
    desv_f = (f - centroid)**2

    spread = np.sqrt((desv_f * S).sum(axis=0) / (S.sum(axis=0)+eps))
    return spread

def spectral_skewness(S,f, centroid = None, spread = None):
    S = np.abs(S)
    eps = 1e-10
    if centroid is None:
        centroid = spectral_centroid(S,f)

    if spread is None:
        spread = spectral_spread(S,f,centroid=centroid)

    centroid = centroid.reshape(1,-1)
    f = f.reshape(-1,1)
 
    desv_f = f - centroid

    num = ((desv_f ** 3)*S).sum(axis=0)
    den = spread**3 * (S.sum(axis=0))
    return num / (den + eps)

def spectral_kurtosis(S,f, centroid = None, spread = None):
    S = np.abs(S)
    eps = 1e-10
    if centroid is None:
        centroid = spectral_centroid(S,f)

    if spread is None:
        spread = spectral_spread(S,f,centroid=centroid)

    centroid = centroid.reshape(1,-1)
    f = f.reshape(-1,1)
 
    desv_f = f - centroid

    num = ((desv_f ** 4)*S).sum(axis=0)
    den = spread**4 * (S.sum(axis=0))
    return num / (den + eps)
    
def extract_features(audio_data, fs):
    win_length = round(0.032*fs)
    hop_length = win_length // 2
    window = 'hamming'
    n_fft = 512
    n_mels = 40
    n_mfcc=13
    f = librosa.core.fft_frequencies(n_fft=n_fft, sr = fs)


    audio_mask = detect_speech(audio_data, fs)
    audio_trim = audio_data[np.where(audio_mask)]
    
    S = librosa.stft(audio_trim, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window)
    absS = np.abs(S)

    Smel = librosa.feature.melspectrogram(S= absS ,sr=fs,  n_fft=n_fft, 
                                      hop_length=hop_length, 
                                      win_length=win_length,n_mels = n_mels,
                                      ) # reutilizamos el espectrograma lineal
    
    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfcc = librosa.feature.mfcc(S = librosa.power_to_db(Smel), sr=fs,  n_mfcc=n_mfcc)

    
    # Calculate delta MFCCs
    
    mfcc_delta = librosa.feature.delta(mfcc)
    
    # Calculate spectral flux
    flux = spectral_flux(absS)

    # Calculate spectral centroid
    centroid = spectral_centroid(absS,f)

    # Calculate spectral spread
    spread = spectral_spread(absS,f,centroid=centroid)

    # Calculate spectral skewness
    skewness = spectral_skewness(absS,f,centroid=centroid, spread=spread)

    # Calculate spectral kurtosis
    kurtosis = spectral_kurtosis(absS,f,centroid=centroid, spread=spread)

    # Calculate spectral rolloff point
    rolloff = librosa.feature.spectral_rolloff(S=absS, sr=fs,  roll_percent=0.85)

    audio_features = {}
    audio_features['mfcc']=mfcc
    audio_features['mfcc_delta']=mfcc_delta
    audio_features['flux']=flux
    audio_features['centroid']=centroid
    audio_features['spread']=spread
    audio_features['skewness']=skewness
    audio_features['kurtosis']=kurtosis
    audio_features['rolloff']=rolloff

    audio_features = vectorize_features(audio_features)
    
    return audio_features


def vectorize_features(audio_features):
    #13 features
    avg_mfcc = np.mean(audio_features['mfcc'],axis=1)
    #13 features
    var_mfcc = np.std(audio_features['mfcc'],axis=1)
    #13 features
    max_mfcc = np.max(audio_features['mfcc'],axis=1)
    #13 features
    min_mfcc = np.min(audio_features['mfcc'],axis=1)
    #13 features
    avg_mfcc_delta = np.mean(audio_features['mfcc_delta'],axis=1)
    #13 features
    var_mfcc_delta = np.std(audio_features['mfcc_delta'],axis=1)

    #2 features
    avg_flux = np.mean(audio_features['flux'])
    var_flux = np.std(audio_features['flux'])

    #5 feature
    avg_centroid = np.mean(audio_features['centroid'])
    avg_spread = np.mean(audio_features['spread'])
    avg_skewness = np.mean(audio_features['skewness'])
    avg_kurtosis = np.mean(audio_features['kurtosis'])
    avg_rolloff = np.mean(audio_features['rolloff'])

                    

    return np.concatenate([avg_mfcc, var_mfcc, 
                           max_mfcc, min_mfcc, 
                           avg_mfcc_delta, var_mfcc_delta,
                           [avg_flux, var_flux],
                           [avg_centroid, avg_spread, avg_skewness, avg_kurtosis, avg_rolloff]])
                           

                           

In [ ]:

import matplotlib.pyplot as plt

* Ejecuta la siguiente celda.
* Pulsa circulo negro para comenzar a grabar.
* Di un numero
* Pulsa de nuevo para terminar la grabacion

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
# convertimos a wav
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -ar 16000 -f wav file.wav  -y -hide_banner -loglevel panic

In [ ]:
# Escucha para comprobar que todo sigue ok
Audio('file.wav')

In [ ]:
# Pintamos para ver el aspecto de la señal  que no sature 
audio_data, fs = librosa.load('file.wav', sr=None)
t = np.arange(len(audio_data))/fs
plt.plot(t,audio_data)
_ = plt.xlabel('time')

In [ ]:
audio_feat = extract_features(audio_data, fs)

In [ ]:
audio_feat_norm = (audio_feat.reshape(1,-1) - m_train)/ s_train

In [ ]:
red.eval()
x = torch.from_numpy(audio_feat_norm.astype(np.float32)).to('cuda')
y = red(x)
print("Has dicho: ", torch.argmax(y).item()) # clasificamos con el máximo